<a href="https://colab.research.google.com/github/zihadbappy/bnfake/blob/main/Final_LSTM_BiLSTM_CNN%2BLSTM_CNN%2BBiLSTM_with_ES.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [151]:
!git clone https://github.com/zihadbappy/bnfakedata.git

fatal: destination path 'bnfakedata' already exists and is not an empty directory.


In [152]:
import pandas as pd
Dataset= pd.read_csv('/content/bnfakedata/bnfakefinal.csv')

In [153]:
df = pd.DataFrame(Dataset)

In [154]:
df = pd.DataFrame(Dataset)


In [155]:
df.groupby('label').agg('count')

,headline,content
label,,
0,2790,2790
1,2790,2789


In [156]:
df=df.dropna()

In [157]:
X=df.content
Y=df.label

In [158]:
Y.value_counts()

0    2790
1    2789
Name: label, dtype: int64

In [159]:
X.shape

(5579,)

In [160]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import Dropout
from keras.callbacks import EarlyStopping

Onehot Representation


In [161]:
voc_size=89000

In [162]:
onehot_repr=[one_hot(words,voc_size)for words in X]

In [163]:
len(onehot_repr)

5579

In [164]:
news_length=300
embedded_news=pad_sequences(onehot_repr,padding='pre',maxlen=news_length)
print(embedded_news)

[[    0     0     0 ... 76144 82749 39018]
 [    0     0     0 ... 40936 43588 48981]
 [ 7286 22395 37647 ... 25418 43588 48981]
 ...
 [    0     0     0 ... 62092 83988 31763]
 [    0     0     0 ...  4096 65258 40023]
 [    0     0     0 ... 80338 22682  6176]]


In [165]:
embedded_news[0]


array([    0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0, 16949,  1149, 88305, 80679, 80420, 10244,
       13973,  9321, 68140, 42081, 43346, 18062, 80893, 24022, 63995,
       71781, 27708, 34945, 36795, 11789, 60517, 49096, 82586, 22208,
       86276, 63995, 53908, 52664, 83533, 27708, 34945, 68246, 47580,
       62094, 10370,  2168, 59073, 20758, 68390,  3100, 30871, 42874,
        9515, 44602, 69083, 36795, 31662, 51058, 78099, 53641, 36795,
       11789, 60517, 74695, 34943, 34945, 71781, 47580,  5038, 20189,
       39816, 39816,

In [166]:
len(embedded_news),Y.shape

(5579, (5579,))

In [167]:
import numpy as np
X_final=np.array(embedded_news)
y_final=np.array(Y)

In [168]:
X_final.shape,y_final.shape

((5579, 300), (5579,))

In [169]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.25, random_state=99)

# **LSTM MODEL**

In [170]:
embedding_vector_features=40
model1=Sequential()
model1.add(Embedding(voc_size,embedding_vector_features,input_length=news_length))
model1.add(LSTM(100))
model1.add(Dense(1,activation='sigmoid'))
model1.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model1.summary())
model1.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=8,batch_size=64,
    callbacks=[EarlyStopping(monitor="val_accuracy",
    min_delta=0,
    patience=3,
    verbose=1,
    mode="auto",
    restore_best_weights=True,)])

Model: "sequential_17"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_17 (Embedding)    (None, 300, 40)           3560000   
                                                                 
 lstm_17 (LSTM)              (None, 100)               56400     
                                                                 
 dense_17 (Dense)            (None, 1)                 101       
                                                                 
Total params: 3,616,501
Trainable params: 3,616,501
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/8
66/66 [==============================] - 4s 30ms/step - loss: 0.4374 - accuracy: 0.8004 - val_loss: 0.2134 - val_accuracy: 0.9326
Epoch 2/8
66/66 [==============================] - 1s 22ms/step - loss: 0.1586 - accuracy: 0.9634 - val_loss: 0.2208 - val_accuracy: 0.9355
Epoch 3/8
66/66 [=====

# **CNN+LSTM MODEL**

In [171]:
from keras.layers import Dense,Conv1D,MaxPooling1D

In [172]:
embedding_vecor_length = 40
model2 = Sequential()
model2.add(Embedding(voc_size, embedding_vecor_length, input_length=news_length))
model2.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
model2.add(MaxPooling1D(pool_size=2))
model2.add(LSTM(100))
model2.add(Dense(1, activation='sigmoid'))
model2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model2.summary())
model2.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=8,batch_size=64,
    callbacks=[EarlyStopping(monitor="val_accuracy",
    min_delta=0,
    patience=3,
    verbose=1,
    mode="auto",
    restore_best_weights=True,)])

Model: "sequential_18"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_18 (Embedding)    (None, 300, 40)           3560000   
                                                                 
 conv1d_8 (Conv1D)           (None, 300, 32)           3872      
                                                                 
 max_pooling1d_8 (MaxPooling  (None, 150, 32)          0         
 1D)                                                             
                                                                 
 lstm_18 (LSTM)              (None, 100)               53200     
                                                                 
 dense_18 (Dense)            (None, 1)                 101       
                                                                 
Total params: 3,617,173
Trainable params: 3,617,173
Non-trainable params: 0
___________________________________________

LSTM and CNN+LSTM evaluation

In [173]:
y_pred1=model1.predict(X_test)
y_pred2=model2.predict(X_test)

In [174]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

In [175]:
print("LSTM accuracy:%.4f"%accuracy_score(y_test,np.around(y_pred1)))
print("CNN+LSTM accuracy:%.4f"%accuracy_score(y_test,np.around(y_pred2)))

LSTM accuracy:0.9520
CNN+LSTM accuracy:0.9534


In [176]:
print("LSTM:\n", classification_report(y_test,np.around(y_pred1),digits=4))
print("\nCNN+LSTM:\n", classification_report(y_test,np.around(y_pred2),digits=4))

LSTM:
               precision    recall  f1-score   support

           0     0.9437    0.9613    0.9524       698
           1     0.9605    0.9426    0.9515       697

    accuracy                         0.9520      1395
   macro avg     0.9521    0.9520    0.9520      1395
weighted avg     0.9521    0.9520    0.9520      1395


CNN+LSTM:
               precision    recall  f1-score   support

           0     0.9689    0.9370    0.9527       698
           1     0.9389    0.9699    0.9541       697

    accuracy                         0.9534      1395
   macro avg     0.9539    0.9534    0.9534      1395
weighted avg     0.9539    0.9534    0.9534      1395



In [177]:
print("LSTM confusion matrix:\n ",confusion_matrix(y_test,np.around(y_pred1)))
print("\nCNN+LSTM confusion matrix:\n",confusion_matrix(y_test,np.around(y_pred2)))

LSTM confusion matrix:
  [[671  27]
 [ 40 657]]

CNN+LSTM confusion matrix:
 [[654  44]
 [ 21 676]]


#**Bi-LSTM MODEL**

In [178]:
embedding_vector_features=40
model3=Sequential()
model3.add(Embedding(voc_size,embedding_vector_features,input_length=news_length))
model3.add(Bidirectional(LSTM(100)))
model3.add(Dropout(0.3))
model3.add(Dense(1,activation='sigmoid'))
model3.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model3.summary())
model3.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=8,batch_size=64,
    callbacks=[EarlyStopping(monitor="val_accuracy",
    min_delta=0,
    patience=3,
    verbose=1,
    mode="auto",
    restore_best_weights=True,)])

Model: "sequential_19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_19 (Embedding)    (None, 300, 40)           3560000   
                                                                 
 bidirectional_8 (Bidirectio  (None, 200)              112800    
 nal)                                                            
                                                                 
 dropout_4 (Dropout)         (None, 200)               0         
                                                                 
 dense_19 (Dense)            (None, 1)                 201       
                                                                 
Total params: 3,673,001
Trainable params: 3,673,001
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/8
66/66 [==============================] - 6s 50ms/step - loss: 0.5010 - accuracy: 0.7615 - val_

## **CNN+BiLSTM MODEL**

In [179]:
embedding_vecor_length = 40
model4 = Sequential()
model4.add(Embedding(voc_size, embedding_vecor_length, input_length=news_length))
model4.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
model4.add(MaxPooling1D(pool_size=2))
model4.add(Bidirectional(LSTM(100)))
model4.add(Dense(1, activation='sigmoid'))
model4.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model4.summary())
model4.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=8,batch_size=64,
    callbacks=[EarlyStopping(monitor="val_accuracy",
    min_delta=0,
    patience=3,
    verbose=1,
    mode="auto",
    restore_best_weights=True,)])

Model: "sequential_20"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_20 (Embedding)    (None, 300, 40)           3560000   
                                                                 
 conv1d_9 (Conv1D)           (None, 300, 32)           3872      
                                                                 
 max_pooling1d_9 (MaxPooling  (None, 150, 32)          0         
 1D)                                                             
                                                                 
 bidirectional_9 (Bidirectio  (None, 200)              106400    
 nal)                                                            
                                                                 
 dense_20 (Dense)            (None, 1)                 201       
                                                                 
Total params: 3,670,473
Trainable params: 3,670,473
N

In [180]:
y_pred3=model3.predict(X_test)
y_pred4=model4.predict(X_test)

In [181]:
print("BiLSTM accuracy:%.4f"%accuracy_score(y_test,np.around(y_pred3)))
print("CNN+BiLSTM accuracy:%.4f"%accuracy_score(y_test,np.around(y_pred4)))

BiLSTM accuracy:0.9606
CNN+BiLSTM accuracy:0.9685


In [182]:
print("BiLSTM:\n",classification_report(y_test,np.around(y_pred3),digits=4))
print("\nCNN+BiLSTM:\n",classification_report(y_test,np.around(y_pred4),digits=4))

BiLSTM:
               precision    recall  f1-score   support

           0     0.9573    0.9642    0.9607       698
           1     0.9639    0.9570    0.9604       697

    accuracy                         0.9606      1395
   macro avg     0.9606    0.9606    0.9606      1395
weighted avg     0.9606    0.9606    0.9606      1395


CNN+BiLSTM:
               precision    recall  f1-score   support

           0     0.9671    0.9699    0.9685       698
           1     0.9698    0.9670    0.9684       697

    accuracy                         0.9685      1395
   macro avg     0.9685    0.9685    0.9685      1395
weighted avg     0.9685    0.9685    0.9685      1395



In [183]:
print("BiLSTM confusion matrix:\n",confusion_matrix(y_test,np.around(y_pred3)))
print("\nCNN+BiLSTM confusion matrix:\n",confusion_matrix(y_test,np.around(y_pred4)))

BiLSTM confusion matrix:
 [[673  25]
 [ 30 667]]

CNN+BiLSTM confusion matrix:
 [[677  21]
 [ 23 674]]
